In [5]:
# change the cell width
from IPython.display import display, HTML
display(HTML("<style>.container { width:110% !important; }</style>"))

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime as dt
import time

In [7]:
sns.set()
sns.set_context(context='talk', font_scale=1.1)

In [8]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [26]:
%%cython
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
import numpy as np
cimport numpy as np
cimport cython

cdef class FwFMScoreSimulator:
    cdef int num_fields, k
    cdef np.float64_t[:, :] vecs   # 2D memoryview for vectors
    cdef np.float64_t[:, :] r       # 2D memoryview for weights

    def __init__(self, int num_fields, int k):
        cdef np.ndarray[np.float64_t, ndim=2] tmp_vecs
        cdef np.ndarray[np.float64_t, ndim=2] tmp_r
        self.num_fields = num_fields
        self.k = k
        tmp_vecs = np.random.normal(size=(num_fields, k)).astype(np.float64)
        self.vecs = tmp_vecs
        tmp_r = np.random.normal(size=(num_fields, num_fields)).astype(np.float64)
        self.r = np.triu(tmp_r)

    @cython.boundscheck(False)
    @cython.wraparound(False)
    def score(self):
        """
        """
        cdef int i, j, k_idx
        cdef double ans = 0.0, dot_val
        for i in range(self.num_fields):
            for j in range(i+1, self.num_fields):
                dot_val = 0.0
                for k_idx in range(self.k):
                    dot_val += self.vecs[i, k_idx] * self.vecs[j, k_idx]
                ans += self.r[i, j] * dot_val
        return ans


In [10]:
%%cython
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
import numpy as np
cimport cython

cdef class TensorFMSimulator:

    cdef int num_fields, l, k
    cdef list dim_int, rank_tensors
    cdef double[:, :] vecs_mv  # memoryview for vecs
    cdef list W_mv             # list of memoryviews for each weight tensor

    def __init__(self, int num_fields, int k, dim_int, rank_tensors):
        cdef int i
        self.num_fields = num_fields
        self.k = k
        self.l = len(dim_int)
        self.dim_int = dim_int
        self.rank_tensors = rank_tensors
        
        self.vecs_mv = np.zeros((num_fields, k), dtype=np.float64)
        
        self.W_mv = []
        for i in range(self.l):
            self.W_mv.append(np.zeros((num_fields, rank_tensors[i], dim_int[i]), dtype=np.float64))


    @cython.boundscheck(False)  # Deactivate bounds checking
    @cython.wraparound(False)   # Deactivate negative indexing.
    def score(self):
        """
        Scoring
        """
        cdef double total = 0.0
        cdef int z, i, j, b, f
        cdef double tmp, dot_val
        # Use the memoryview for vecs (which is C-contiguous in our case)
        cdef double[:, :] vecs = self.vecs_mv
        cdef double[:,:,:] W  # will point to each weight tensor



        for z in range(self.l):  
            W = self.W_mv[z]  
            for i in range(self.k):
                for j in range(self.rank_tensors[z]):
                    tmp = 1.0
                    for b in range(self.dim_int[z]):
                        dot_val = 0.0
                        for f in range(self.num_fields):
                            dot_val += W[f, j, b] * vecs[f, i]
                        tmp *= dot_val
                    total += tmp
        return total


In [ ]:
%%cython
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
import numpy as np
cimport numpy as np
cimport cython

cdef class HOFM3ScoreSimulator:
    """
    Simulator for a 3rd-order HOFM-style score for a single example,
    consistent with the original PyTorch implementation:

        y = FM_2(x2) + ANOVA_3(x3)

    where:
      - FM_2 replicates torchfm.layer.FactorizationMachine(reduce_sum=True)
      - ANOVA_3 replicates AnovaKernel(order=3, reduce_sum=True)

    We simulate the embedding matrices x2 and x3 with random values.
    """

    cdef int num_fields, k
    cdef np.float64_t[:, :] vecs2   # embeddings for 2nd-order FM
    cdef np.float64_t[:, :] vecs3   # embeddings for 3rd-order ANOVA

    def __init__(self, int num_fields, int k):
        cdef np.ndarray[np.float64_t, ndim=2] tmp_vecs2
        cdef np.ndarray[np.float64_t, ndim=2] tmp_vecs3
        self.num_fields = num_fields
        self.k = k

        # This mirrors: embedding of size (num_fields, embed_dim) per order.
        tmp_vecs2 = np.random.normal(size=(num_fields, k)).astype(np.float64)
        tmp_vecs3 = np.random.normal(size=(num_fields, k)).astype(np.float64)
        self.vecs2 = tmp_vecs2
        self.vecs3 = tmp_vecs3

    @cython.boundscheck(False)
    @cython.wraparound(False)
    def score(self):
        """
        Compute:
          y = FactorizationMachine(vecs2) + AnovaKernel(order=3)(vecs3)
        for a single instance (no batch dimension).
        """
        cdef int i, j, f, t
        cdef double ans = 0.0
        cdef double val, s1, s2

        # ----- 2nd-order FM term (FactorizationMachine) -----
        # torch code (for single example) is essentially:
        #   square_of_sum = (sum_i x[i])^2
        #   sum_of_square = sum_i x[i]^2
        #   ix = square_of_sum - sum_of_square
        #   fm = 0.5 * sum_f ix[f]
        for f in range(self.k):
            s1 = 0.0   # sum_i v2[i,f]
            s2 = 0.0   # sum_i v2[i,f]^2
            for i in range(self.num_fields):
                val = self.vecs2[i, f]
                s1 += val
                s2 += val * val
            ans += 0.5 * (s1 * s1 - s2)

        # ----- 3rd-order ANOVA term (AnovaKernel(order=3)) -----
        #
        # PyTorch code (per batch) does:
        #   a_prev = ones(batch, F+1, k)
        #   for t in range(order):
        #       a = zeros(batch, F+1, k)
        #       a[:, t+1:, :] += x[:, t:, :] * a_prev[:, t:-1, :]
        #       a = cumsum(a, dim=1)
        #       a_prev = a
        #   out = sum(a_prev[:, -1, :], dim=-1, keepdim=True)
        #
        # Here we replicate that for a single example (no batch):
        #
        cdef np.ndarray[np.float64_t, ndim=2] a_prev_np = \
            np.ones((self.num_fields + 1, self.k), dtype=np.float64)
        cdef np.ndarray[np.float64_t, ndim=2] a_np = \
            np.zeros((self.num_fields + 1, self.k), dtype=np.float64)

        cdef np.float64_t[:, :] a_prev = a_prev_np
        cdef np.float64_t[:, :] a = a_np

        # x is "vecs3" with shape (num_fields, k)
        cdef np.float64_t[:, :] x = self.vecs3

        for t in range(3):  # order = 3
            # a = 0
            for j in range(self.num_fields + 1):
                for f in range(self.k):
                    a[j, f] = 0.0

            # a[t+1:, :] += x[t:, :] * a_prev[t:-1, :]
            # note: in PyTorch, x[:, t:, :] and a_prev[:, t:-1, :]
            # become x[t:] and a_prev[t:-1] here (single example).
            for j in range(t + 1, self.num_fields + 1):
                # j runs from t+1 to num_fields
                # correspond to x index j-1 and a_prev index j-1
                for f in range(self.k):
                    a[j, f] += x[j - 1, f] * a_prev[j - 1, f]

            # a = cumsum(a, dim=1) along the field dimension
            for j in range(1, self.num_fields + 1):
                for f in range(self.k):
                    a[j, f] += a[j - 1, f]

            # a_prev = a
            for j in range(self.num_fields + 1):
                for f in range(self.k):
                    a_prev[j, f] = a[j, f]

        # reduce_sum=True: sum over embed_dim of a_prev[-1, :]
        for f in range(self.k):
            ans += a_prev[self.num_fields, f]

        return ans


Error compiling Cython file:
------------------------------------------------------------
...

    We simulate the embedding matrices x2 and x3 with random values.
    """

    cdef int num_fields, k
    cdef np.float64_t[:, :] vecs2   # embeddings for 2nd-order FM
         ^
------------------------------------------------------------

/Users/amazzetto/.ipython/cython/_cython_magic_e3af113d51b4bd7c36fe14c00a344494db542574.pyx:20:9: 'np' is not a cimported module

Error compiling Cython file:
------------------------------------------------------------
...
    We simulate the embedding matrices x2 and x3 with random values.
    """

    cdef int num_fields, k
    cdef np.float64_t[:, :] vecs2   # embeddings for 2nd-order FM
    cdef np.float64_t[:, :] vecs3   # embeddings for 3rd-order ANOVA
         ^
------------------------------------------------------------

/Users/amazzetto/.ipython/cython/_cython_magic_e3af113d51b4bd7c36fe14c00a344494db542574.pyx:21:9: 'np' is not a cimported m

In [12]:
%%cython
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
import numpy as np
cimport numpy as np
cimport cython

cdef class CrossNetworkWithLinearSimulator:
    """
    Simulates a CrossNetwork layer (or layers) followed by a linear layer.
    
    The simulator holds its input vector as an attribute (self.x0), similar to the original FwFM code.
    For an input vector x0 of length `input_dim`, each cross layer performs:
         xw = dot(x, w)           (a scalar)
         x = x + x0 * xw + b       (elementwise update)
    
    After processing all cross layers, a linear layer transforms the output vector x into a scalar:
         p = dot(x, linear_w) + linear_b
         
    The score() method computes and returns the scalar output.
    """
    cdef int input_dim, num_layers
    cdef list W_mv  # List of weight vectors for each cross layer (each is a NumPy array of shape (input_dim,))
    cdef list b_mv  # List of bias vectors for each cross layer (each is a NumPy array of shape (input_dim,))
    
    # Parameters for the linear layer (from input_dim -> 1)
    cdef np.ndarray linear_w  # Weight vector (shape: (input_dim,))
    cdef double linear_b      # Bias scalar
    
    # The input vector is stored as part of the simulator.
    cdef np.ndarray x0  # Input vector (shape: (input_dim,))
    
    def __init__(self, int input_dim, int num_layers):
        self.input_dim = input_dim
        self.num_layers = num_layers
        
        # Initialize cross network parameters.
        self.W_mv = []
        self.b_mv = []
        cdef int l
        for l in range(num_layers):
            # For simulation, we initialize weights and biases as zeros.
            # You may choose to initialize with random values if desired.
            self.W_mv.append(np.zeros(input_dim, dtype=np.float64))
            self.b_mv.append(np.zeros(input_dim, dtype=np.float64))
            
        # Initialize the linear layer parameters.
        self.linear_w = np.zeros(input_dim, dtype=np.float64)
        self.linear_b = 1.0
        
        # Initialize the input vector; this can be updated externally as needed.
        self.x0 = np.zeros(input_dim, dtype=np.float64)
    
    @cython.boundscheck(False)
    @cython.wraparound(False)
    def score(self):
        """
        Computes the scalar output using the stored input (self.x0).
        No parameters are accepted.
        """
        cdef int i, l, n = self.input_dim
        cdef double xw, p = 0.0
        
        # Copy the stored input vector (x0) into x. This vector will be updated.
        cdef np.ndarray[np.float64_t, ndim=1] x_arr = self.x0.copy()
        cdef double[:] x = x_arr
        
        # Create a memoryview for the original input.
        cdef double[:] x0_mv = self.x0
        
        # Declare temporary variables for layer weights and biases outside the loop.
        cdef np.ndarray[np.float64_t, ndim=1] w_arr, b_arr
        cdef double[:] w, b
        
        # Loop over cross layers.
        for l in range(self.num_layers):
            # Assign the current layer's weight and bias arrays to the pre-declared variables.
            w_arr = self.W_mv[l]
            b_arr = self.b_mv[l]
            w = w_arr  # Create a memoryview for weights.
            b = b_arr  # Create a memoryview for biases.
            
            # Compute the dot product between x and w.
            xw = 0.0
            for i in range(n):
                xw += x[i] * w[i]
            
            # Update x elementwise: x = x + x0 * xw + b.
            for i in range(n):
                x[i] = x[i] + x0_mv[i] * xw + b[i]
                
        # Apply the linear layer: compute p = dot(x, linear_w) + linear_b.
        cdef np.ndarray[np.float64_t, ndim=1] lw_arr = self.linear_w
        cdef double[:] lw = lw_arr
        for i in range(n):
            p += x[i] * lw[i]
        p += self.linear_b
        
        return p

In [13]:
%%cython
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport exp

cdef double relu(double x):
    return x if x > 0 else 0.0

cdef class AFMScoreSimulator:
    cdef int num_fields, embed_dim, attn_size
    cdef np.float64_t[:, :, :] x                      # shape: (1, num_fields, embed_dim)
    cdef np.float64_t[:, :] attention_weights         # shape: (embed_dim, attn_size)
    cdef np.float64_t[:, :] projection_weights        # shape: (attn_size, 1)
    cdef np.float64_t[:] fc_weights                   # shape: (embed_dim,)

    def __init__(self, int num_fields, int embed_dim, int attn_size):
        self.num_fields = num_fields
        self.embed_dim = embed_dim
        self.attn_size = attn_size

        self.x = np.random.normal(size=(1, num_fields, embed_dim)).astype(np.float64)
        self.attention_weights = np.random.normal(size=(embed_dim, attn_size)).astype(np.float64)
        self.projection_weights = np.random.normal(size=(attn_size, 1)).astype(np.float64)
        self.fc_weights = np.random.normal(size=(embed_dim,)).astype(np.float64)

    @cython.boundscheck(False)
    @cython.wraparound(False)
    def score(self):
        cdef int i, j, k, h, idx
        cdef int pair_count = (self.num_fields * (self.num_fields - 1)) // 2
        cdef np.float64_t[:, :] inner_product = np.zeros((pair_count, self.embed_dim))
        cdef np.float64_t[:, :] attn_hidden = np.zeros((pair_count, self.attn_size))
        cdef np.float64_t[:] attn_raw = np.zeros(pair_count)
        cdef np.float64_t[:] attn_score = np.zeros(pair_count)
        cdef np.float64_t[:] attn_output = np.zeros(self.embed_dim)
        cdef double val, sum_exp = 0.0

        # Step 1: pairwise elementwise products
        idx = 0
        for i in range(self.num_fields - 1):
            for j in range(i + 1, self.num_fields):
                for k in range(self.embed_dim):
                    inner_product[idx, k] = self.x[0, i, k] * self.x[0, j, k]
                idx += 1

        # Step 2: attention hidden layer (ReLU(attention(inner_product)))
        for idx in range(pair_count):
            for h in range(self.attn_size):
                val = 0.0
                for k in range(self.embed_dim):
                    val += inner_product[idx, k] * self.attention_weights[k, h]
                attn_hidden[idx, h] = relu(val)

        # Step 3: projection to scalar per interaction
        for idx in range(pair_count):
            val = 0.0
            for h in range(self.attn_size):
                val += attn_hidden[idx, h] * self.projection_weights[h, 0]
            attn_raw[idx] = val

        # Step 4: softmax over interactions
        for idx in range(pair_count):
            attn_score[idx] = exp(attn_raw[idx])
            sum_exp += attn_score[idx]
        for idx in range(pair_count):
            attn_score[idx] /= sum_exp

        # Step 5: weighted sum of inner products
        for idx in range(pair_count):
            for k in range(self.embed_dim):
                attn_output[k] += attn_score[idx] * inner_product[idx, k]

        # Step 6: final projection
        val = 0.0
        for k in range(self.embed_dim):
            val += attn_output[k] * self.fc_weights[k]
        return val


In [14]:
results = []
score_count = 100000
experiment_repeat = 1

In [15]:
class Benchmark:
    def __init__(self, results, model, ranks, dim_int, count):
        self.results = results
        self.model = model
        self.ranks = ranks
        self.dim_int =  dim_int
        self.count = count
    
    # enter the context manager
    def __enter__(self):
        self.time_start = time.perf_counter()
        return self

    def __exit__(self, type, value, traceback):
        duration = time.perf_counter() - self.time_start

        result = {
            'Model': self.model,
            'Rank': self.ranks,
            'Interactions' : self.dim_int,
            'Scoring time (ms)': 1000 * duration / self.count
        }
        self.results.append(result)        
        return self

In [16]:
# compute low rank timings
for order_interactions in [[2],[2,3],[2,3,4]]:
    for rank in [1,2,4]:
        x = dt.now().isoformat()
        print(f'Current ISO time: {x}, rank = {rank}', 'order interactions = {order_interactions}')
        for i in range(experiment_repeat):
            tensorfm_sim = TensorFMSimulator(num_fields=100, k=8, rank_tensors = [rank]*len(order_interactions), dim_int = order_interactions )
            with Benchmark(results, 'TensorFM', [rank]*len(order_interactions), order_interactions, score_count) as benchmark:
                for j in range(score_count):
                    tensorfm_sim.score()

Current ISO time: 2026-02-12T23:20:35.078941, rank = 1 order interactions = {order_interactions}
Current ISO time: 2026-02-12T23:20:35.163486, rank = 2 order interactions = {order_interactions}
Current ISO time: 2026-02-12T23:20:35.316381, rank = 4 order interactions = {order_interactions}
Current ISO time: 2026-02-12T23:20:35.605749, rank = 1 order interactions = {order_interactions}
Current ISO time: 2026-02-12T23:20:35.804431, rank = 2 order interactions = {order_interactions}
Current ISO time: 2026-02-12T23:20:36.175293, rank = 4 order interactions = {order_interactions}
Current ISO time: 2026-02-12T23:20:36.931302, rank = 1 order interactions = {order_interactions}
Current ISO time: 2026-02-12T23:20:37.290157, rank = 2 order interactions = {order_interactions}
Current ISO time: 2026-02-12T23:20:37.981023, rank = 4 order interactions = {order_interactions}


In [17]:
# compute FwFM timings
x = dt.now().isoformat()
print(f'Current ISO time: {x}')
for i in range(experiment_repeat):
    fwfm_sim = FwFMScoreSimulator(num_fields=100, k=8)
    with Benchmark(results, 'FwFM', None, None, score_count) as benchmark:
        for j in range(score_count):
            x = fwfm_sim.score()

Current ISO time: 2026-02-12T23:20:39.317039


In [18]:
x = dt.now().isoformat()
print(f'Current ISO time: {x}')
for i in range(experiment_repeat):
    cn_sim = CrossNetworkWithLinearSimulator(input_dim=100*8,num_layers=3)
    with Benchmark(results, 'CN', None, None, score_count) as benchmark:
        for j in range(score_count):
            x = cn_sim.score()

Current ISO time: 2026-02-12T23:20:40.618582


In [19]:
x = dt.now().isoformat()
print(f'Current ISO time: {x}')
for i in range(experiment_repeat):
    afm_sim = AFMScoreSimulator(num_fields=100,embed_dim=8,attn_size=8)
    with Benchmark(results, 'AFM', None, None, score_count) as benchmark:
        for j in range(score_count):
            x = afm_sim.score()

Current ISO time: 2026-02-12T23:20:41.091930


In [20]:
x = dt.now().isoformat()
print(f'Current ISO time: {x}')
for i in range(experiment_repeat):
    sim_hofm = HOFM3ScoreSimulator(num_fields=100,k=8)
    with Benchmark(results, 'HOFM', None, None, score_count) as benchmark:
        for j in range(score_count):
            x = sim_hofm.score()

Current ISO time: 2026-02-12T23:21:01.507124


NameError: name 'HOFM3ScoreSimulator' is not defined

In [ ]:
results = pd.DataFrame.from_records(results)
results

,Model,Rank,Interactions,Scoring time (ms)
0,TensorFM,[1],[2],0.001147
1,TensorFM,[2],[2],0.001520
2,TensorFM,[4],[2],0.002857
3,TensorFM,"[1, 1]","[2, 3]",0.001939
4,TensorFM,"[2, 2]","[2, 3]",0.003671
5,TensorFM,"[4, 4]","[2, 3]",0.007025
6,TensorFM,"[1, 1, 1]","[2, 3, 4]",0.003402
7,TensorFM,"[2, 2, 2]","[2, 3, 4]",0.006533
8,TensorFM,"[4, 4, 4]","[2, 3, 4]",0.012923
9,FwFM,None,None,0.012647


In [ ]:
sns.relplot(data=results, x='# context fields', y='Scoring time (ms)', hue='Model', col='Rank', kind='line', facet_kws=dict(sharex=True, sharey=False))

ValueError: Could not interpret value `# context fields` for `x`. An entry with this name does not appear in `data`.